In [ ]:
!pip install BisPy
!pip install ogb
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for llist: filename=llist-0.7.1-cp310-cp310-linux_x86_64.whl size=74085 sha256=6987dba6b2821d89cf3e84baee7ce3452b1cb9a44dcf2e7c73e94866c4664140
  Stored in directory: /root/.cache/pip/wheels/e9/91/41/252becb26acf296e14913e0ee13940295bc475be57aed9bb2e
Successfully built llist
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7026 sha256=55a4a2195e16afad6ed26ebca529869e3e71c37a1049437451cce6bdefa3ebcb
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.2 MB/s eta 0:00:00


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import pandas as pd
from torch_geometric.datasets import Planetoid, CoraFull, Yelp, WebKB, Actor
import os
import bispy
from bispy import compute_maximum_bisimulation, Algorithms
from bispy.dovier_piazza_policriti.dovier_piazza_policriti import dovier_piazza_policriti_partition
from bispy.dovier_piazza_policriti.ranked_partition import RankedPartition
from bispy.dovier_piazza_policriti.dovier_piazza_policriti import dovier_piazza_policriti, collapse, build_block_counterimage, split_upper_ranks
from bispy.utilities.rank_computation import compute_rank
from bispy.utilities.graph_decorator import decorate_nx_graph, decorate_bispy_graph,as_bispy_graph,to_tuple_list
from bispy.utilities import *
import numpy as np
from sklearn.cluster import KMeans
from collections import Counter
import random
import copy
import time
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import matplotlib.pyplot as plt
import numpy as np
from ogb.nodeproppred import PygNodePropPredDataset
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
from sklearn.metrics.pairwise import cosine_similarity

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pickle
from torch_geometric.data import Data
import os
# Load the graph data from the .pkl file using pickle
with open('gdrive/My Drive/GNN_Sparsification/ogbn-products.pkl', 'rb') as f:
    data = pickle.load(f)


model_save_path_GCN = os.path.join('gdrive/My Drive/GNN_Sparsification/Pre_Trained_5%_ogbn_products_GCN.pt')
model_save_path_GAT = os.path.join('gdrive/My Drive/GNN_Sparsification/Pre_Trained_5%_ogbn_products_GAT.pt')
model_save_path_GraphSage = os.path.join('gdrive/My Drive/GNN_Sparsification/Pre_Trained_5%_ogbn_products_GraphSage.pt')
data

Data(num_nodes=1836106, edge_index=[2, 69523198], x=[1836106, 100], y=[1836106])

In [ ]:
import pickle
import networkx as nx

# Replace 'your_graph_file.pkl' with the actual path to your Pickle file
file_path = 'gdrive/My Drive/GNN_Sparsification/5%_Induced_SubG_ogbn_products.pkl'
# Open the Pickle file in binary read mode
with open(file_path, 'rb') as file:
    # Load the graph object from the Pickle file
    loaded_graph = pickle.load(file)


In [ ]:
# Convert node labels to integers with continuous numbers
# Convert node labels to integers with continuous numbers
H = nx.convert_node_labels_to_integers(loaded_graph, first_label=0, ordering='default', label_attribute=None)

mapping = dict(enumerate(loaded_graph.nodes()))

# Print the mapping dictionary
print("Node Mapping Dictionary:", mapping)

Node Mapping Dictionary: 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# Load tensor from .pt file
# loaded_tensor = torch.load('gdrive/My Drive/GNN_Sparsification/k_spanner_100.pt')
loaded_tensor = torch.load('gdrive/My Drive/GNN_Sparsification/5%_dir_induced_yelp_sparsified_0.25.pt')
# loaded_tensor = torch.load('gdrive/My Drive/GNN_Sparsification/Baselines/Dspar_INDUCED/10%/Arxiv/10%_dir_induced_arxiv_sparsified_0.35.pt')
# loaded_tensor = torch.load('gdrive/My Drive/GNN_Sparsification/Baselines/D_spar_INDUCED/10%/10%_induced_arxiv_sparsified_0.35.pt')
# loaded_tensor = torch.load('gdrive/My Drive/GNN_Sparsification/Baselines/K_spanner_INDUCED/10%/2_spanner.pt')
# data.edge_index = loaded_tensor
# data
loaded_tensor.edge_index.shape


torch.Size([2, 678096])

In [ ]:
# loaded_tensor = torch.load('gdrive/My Drive/GNN_Sparsification/arxiv_sparsified_1.55.pt')
# loaded_tensor

loaded_tensor = torch.load('gdrive/My Drive/GNN_Sparsification/MAG_8_0.8_1.05_Dspar.pt')
loaded_tensor

Data(x=[173559, 800], y=[173559], edge_index=[2, 75302], num_nodes=173559, edge_attr=[75302])

In [ ]:
loaded_tensor = torch.load('gdrive/My Drive/GNN_Sparsification/5%_dir_induced_yelp_sparsified_0.25.pt')
loaded_tensor

Data(edge_index=[2, 678096], num_nodes=269250, edge_attr=[678096])

In [ ]:
from collections import Counter
import random
import copy

new_x = torch.zeros(len(list(G.nodes)),data.x.shape[1])
new_y = torch.zeros(len(list(G.nodes)))

for i in range(len(list(G.nodes))):
  new_x[i,:] = data.x[mapping[i],:]
  new_y[i] = int(data.y[mapping[i]])

# edges = list(G.edges())

# # Create a dictionary to map node labels to integer indices
# node_mapping = {node: index for index, node in enumerate(G.nodes())}

# # Convert the edges to indices using the node_mapping
# new_edge_index = [[node_mapping[source], node_mapping[target]] for source, target in edges]
# new_edge_index = torch.tensor(new_edge_index, dtype=torch.long).t().contiguous()


data_new1 = copy.copy(data)
data_new1.x = new_x
data_new1.y = new_y.long()
data_new1.edge_index = loaded_tensor
data_new1.num_nodes = len(new_y)

NameError: ignored

In [ ]:
data_new1

Data(num_nodes=94773, edge_index=[2, 208418], x=[94773, 128], node_year=[169343, 1], y=[94773])

In [ ]:
from torch_geometric.utils import is_undirected

# Example edge_list representing edges of a graph
edge_list = data.edge_index

# Convert edge_list to a PyTorch Geometric data object (edge_index)
edge_index = torch.t(edge_list)

# Check if the graph is undirected
is_undirected_graph = is_undirected(edge_index)
print("Is the graph undirected?", is_undirected_graph)

Is the graph undirected? False


In [ ]:
import copy
from torch_geometric.utils import subgraph
train_mask = torch.rand(data.num_nodes) < 0.9
test_mask = ~train_mask
tested_elements =  torch.nonzero(test_mask==True).squeeze()
tested_elements = tested_elements.tolist()

# file_path = "gdrive/My Drive/GNN_Sparsification/5%_ogbn_arxiv_train_mask.pt"

# file_path = "gdrive/My Drive/GNN_Sparsification/10%_ogbn_product_train_mask.pt"

file_path = "gdrive/My Drive/GNN_Sparsification/10%_reddit_train_mask.pt"

# Save the binary tensor mask to a .pt file
torch.save(train_mask, file_path)

In [ ]:
# train_mask = torch.load('gdrive/My Drive/GNN_Sparsification/10%_ogbn_arxiv_train_mask.pt')
# train_mask = torch.load("gdrive/My Drive/GNN_Sparsification/10%_ogbn_product_train_mask.pt")
# train_mask = torch.load("gdrive/My Drive/GNN_Sparsification/10%_yelp_train_mask.pt")
train_mask = torch.load('gdrive/My Drive/GNN_Sparsification/5%_ogbn_products_train_mask.pt')
test_mask = ~train_mask
tested_elements =  torch.nonzero(test_mask==True).squeeze()
tested_elements = tested_elements.tolist()

In [ ]:
train_mask

tensor([True, True, True,  ..., True, True, True])

In [ ]:
# Convert PyG graph to NetworkX graph
def pyg_to_networkx(pyg_data):
    edge_index = pyg_data.edge_index
    num_nodes = pyg_data.num_nodes
    edge_list = edge_index.t().tolist()
    G = nx.DiGraph()
    G.add_nodes_from(range(num_nodes))
    G.add_edges_from(edge_list)
    return G

networkx_graph = pyg_to_networkx(data)

In [ ]:
num_nodes = networkx_graph.number_of_nodes()
num_edges = networkx_graph.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

# partition = compute_maximum_bisimulation(networkx_graph)
# len(partition)

Number of nodes: 1836106
Number of edges: 69523106


# Radius-based K-hops Induced Subgraph

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Define two tensors as NumPy arrays
tensor1 = np.array([1, 2, 3])
tensor2 = np.array([4, 5, 6])

# Reshape the tensors into 2D arrays for cosine_similarity function
tensor1 = tensor1.reshape(1, -1)
tensor2 = tensor2.reshape(1, -1)

# Calculate cosine similarity
similarity = cosine_similarity(data.x[tested_elements[2]].numpy().reshape(1, -1), data.x[tested_elements[1000]].numpy().reshape(1, -1))

print("Cosine Similarity using scikit-learn:", similarity[0][0])

Cosine Similarity using scikit-learn: 0.5655823


In [ ]:
import time
import networkx as nx

# Measure the execution time
start_time = time.time()


nodes_to_find_neighbors = tested_elements
neighbors = set()
up_to_k_hop_neighbors = []
up_to_k_hop_neighbors_list = []
k = 1
# G = nx.Graph()
G = nx.DiGraph()

for i in range(k):
  # Iterate through the nodes in your set
  print(i)
  neighbors = set()
  for node in nodes_to_find_neighbors:
    # Use the neighbors() function to get 1-hop neighbors and add them to the set
    neighbors.update(networkx_graph.neighbors(node))
    neighbors.update(networkx_graph.predecessors(node))

    if i == 0:
      G.add_node(node)

      for neighbor in networkx_graph.neighbors(node):
        # if neighbor not in nodes_to_find_neighbors:
        G.add_edge(node, neighbor)

      # for neighbor in networkx_graph.predecessors(node):
      #   # if neighbor not in nodes_to_find_neighbors:
      #   G.add_edge(neighbor, node)


    else:
      for neighbor in networkx_graph.neighbors(node):
        if neighbor not in up_to_k_hop_neighbors_list:
          G.add_edge(node, neighbor)

      for neighbor in networkx_graph.predecessors(node):
        if neighbor not in up_to_k_hop_neighbors_list:
          G.add_edge(neighbor, node)




  # ensure the disjoint property
  if i > 0:
    for ele in up_to_k_hop_neighbors:
      neighbors_list = list(set(list(neighbors)).difference(set(ele)))
    neighbors_list = list(set(neighbors).difference(set(tested_elements)))

  if i == 0:
    neighbors_list = list(set(neighbors).difference(set(tested_elements)))
    up_to_k_hop_neighbors_list += tested_elements

  nodes_to_find_neighbors = neighbors_list
  up_to_k_hop_neighbors.append(neighbors_list)
  up_to_k_hop_neighbors_list += neighbors_list


end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

0
Execution time: 11.094032287597656 seconds


In [ ]:
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

import time

# Measure the execution time
start_time = time.time()

partition = compute_maximum_bisimulation(G)
len(partition)

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

Number of nodes: 1105848
Number of edges: 3525068
Execution time: 73.68832206726074 seconds


In [ ]:
len(tested_elements)

91998

In [ ]:
import pickle
# Assuming you have a NetworkX graph called 'graph'
with open("gdrive/My Drive/GNN_Sparsification/10%_Induced_SubG_Yelp.pkl", "wb") as f:
    pickle.dump(G, f)

In [ ]:
import pickle
# Assuming you have a NetworkX graph called 'graph'
with open("gdrive/My Drive/GNN_Sparsification/10%_Induced_SubG_Ogbn_Product.pkl", "wb") as f:
    pickle.dump(G, f)

In [ ]:
import pickle
# Assuming you have a NetworkX graph called 'graph'
with open("gdrive/My Drive/GNN_Sparsification/15%_Induced_SubG_Ogbn_Arxiv.pkl", "wb") as f:
    pickle.dump(G, f)

In [ ]:
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

partition = compute_maximum_bisimulation(G)
len(partition)

Number of nodes: 380057
Number of edges: 1387625


24633

In [ ]:
len(tested_elements)+len(neighbors)

94699

In [ ]:
def find_tuple_index_with_integer(list_of_tuples, integer):
    return next((index for index, tup in enumerate(list_of_tuples) if integer in tup), -1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import euclidean
from scipy.cluster.hierarchy import linkage, fcluster
import numpy as np
from collections import defaultdict

def exclude_tested_elements_from_MB(parition, tested_elements):

    element_indices = defaultdict(lambda: -1)
    indices = []
    tuple_list = partition

    # Sort the list by the number of elements in each tuple
    sorted_list = sorted(tuple_list, key=lambda x: len(x), reverse=True)
    list_of_lists = [list(t) for t in sorted_list]

    for idx, ele_list in enumerate(list_of_lists):
        for ele in ele_list:
            if element_indices[ele] == -1:
                element_indices[ele] = idx

    # Process tested_elements in chunks
    chunk_size = 5000  # Adjust the chunk size based on available memory
    for i in range(0, len(tested_elements), chunk_size):
        chunk = tested_elements[i:i + chunk_size]
        for ele in chunk:
            id = element_indices[ele]
            if len(list_of_lists[id]) > 1:
                list_of_lists[id].remove(ele)
                list_of_lists.append([ele])
                indices.append(len(list_of_lists) - 1)
            else:
                indices.append(id)

    return list_of_lists, indices

def alpha_Sparsification(list_of_lists, data, alpha):
  list_of_tuples = [tuple(inner_list) for inner_list in list_of_lists]
  i = 0
  while len(list_of_lists[i])>1:

      Z = linkage(data.x[list_of_tuples[i],:], method='average', metric='cosine')
      threshold = alpha * np.max(Z[:, 2])
      clusters = fcluster(Z, threshold, criterion='distance')

      # Get cluster labels for each data point
      labels = clusters


      # Z = linkage(data.x[list_of_tuples[i],:], method='average', metric='euclidean')
      # threshold = alpha * np.max(Z[:, 2])
      # clusters = fcluster(Z, threshold, criterion='distance')

      # # Get cluster labels for each data point
      # labels = clusters




      # Sample tuple to split
      data_tuple = list_of_tuples[i]

      # Create a dictionary to store split data
      split_data = {}

      # Iterate through the labels and tuple elements
      for label, value in zip(labels, data_tuple):
        if label not in split_data:
          split_data[label] = []
        split_data[label].append(value)

      # Convert the dictionary to a list of tuples (if needed)
      split_tuples = [tuple(values) for key, values in split_data.items()]
      list_of_tuples.pop(i)
      j = 0
      for values in split_tuples:
        if j == 0:
          list_of_tuples.insert(0, values)
        else:
          list_of_tuples.insert(len(list_of_tuples), values)
        j+=1

      i+=1

  result_dict = {element: index for index, tup in enumerate(list_of_tuples) for element in tup}

  return list_of_tuples, result_dict

In [ ]:
len(G.nodes())

1362512

In [ ]:
data

Data(num_nodes=1836106, edge_index=[2, 69523198], x=[1836106, 100], y=[1836106])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import euclidean
from scipy.cluster.hierarchy import linkage, fcluster
import numpy as np

def exclude_tested_elements_from_MB(parition, tested_elements):
  tuple_list = partition

  # Sort the list by the number of elements in each tuple
  sorted_list = sorted(tuple_list, key=lambda x: len(x), reverse=True)

  indices = []
  list_of_lists = [list(t) for t in sorted_list]

  for ele in tested_elements:
    id = find_tuple_index_with_integer(sorted_list, ele)
    if len(list_of_lists[id]) > 1:
      list_of_lists[id].remove(ele)
      list_of_lists.append([ele])
      indices.append(len(list_of_lists)-1)
    else:
      indices.append(id)

  return list_of_lists, indices

def alpha_Sparsification(list_of_lists, data, alpha):
  list_of_tuples = [tuple(inner_list) for inner_list in list_of_lists]
  i = 0
  while len(list_of_lists[i])>1:

      Z = linkage(data.x[list_of_tuples[i],:], method='average', metric='cosine')
      threshold = alpha * np.max(Z[:, 2])
      clusters = fcluster(Z, threshold, criterion='distance')

      # Get cluster labels for each data point
      labels = clusters


      # Z = linkage(data.x[list_of_tuples[i],:], method='average', metric='euclidean')
      # threshold = alpha * np.max(Z[:, 2])
      # clusters = fcluster(Z, threshold, criterion='distance')

      # # Get cluster labels for each data point
      # labels = clusters




      # Sample tuple to split
      data_tuple = list_of_tuples[i]

      # Create a dictionary to store split data
      split_data = {}

      # Iterate through the labels and tuple elements
      for label, value in zip(labels, data_tuple):
        if label not in split_data:
          split_data[label] = []
        split_data[label].append(value)

      # Convert the dictionary to a list of tuples (if needed)
      split_tuples = [tuple(values) for key, values in split_data.items()]
      list_of_tuples.pop(i)
      j = 0
      for values in split_tuples:
        if j == 0:
          list_of_tuples.insert(0, values)
        else:
          list_of_tuples.insert(len(list_of_tuples), values)
        j+=1

      i+=1

  result_dict = {element: index for index, tup in enumerate(list_of_tuples) for element in tup}

  return list_of_tuples, result_dict

In [ ]:
# Function to split long tuples
def split_long_tuples(lst, max_length):
    new_list = []
    for tup in lst:
        if len(tup) > max_length:
            # Split the tuple into smaller tuples
            split_tuples = [tup[i:i+max_length] for i in range(0, len(tup), max_length)]
            new_list.extend(split_tuples)
        else:
            new_list.append(tup)
    return new_list

# Split tuples with length over 10K into tuples with length 10K or under
max_length = 500
partition = split_long_tuples(partition, max_length)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import euclidean
from scipy.cluster.hierarchy import linkage, fcluster
import numpy as np

def exclude_tested_elements_from_MB(partition, tested_elements):
    tuple_dict = {}
    indices = []

    tuple_list = partition

    # Sort the list by the number of elements in each tuple
    sorted_list = sorted(tuple_list, key=lambda x: len(x), reverse=True)

    list_of_lists = [list(t) for t in sorted_list]

    # Convert tuples to lists and create a dictionary with elements as keys and their corresponding tuple index as values
    for idx, tpl in enumerate(list_of_lists):
        tpl_list = list(tpl)
        for ele in tpl_list:
            tuple_dict[ele] = idx
        list_of_lists[idx] = tpl_list

    # Update the dictionary and indices based on tested elements
    for ele in tested_elements:
        if ele in tuple_dict:
            idx = tuple_dict[ele]
            if len(list_of_lists[idx]) > 1:
                list_of_lists[idx].remove(ele)
                list_of_lists.append([ele])
                tuple_dict[ele] = len(list_of_lists) - 1
                indices.append(len(list_of_lists) - 1)
            else:
                indices.append(idx)

    return list_of_lists, indices

def alpha_Sparsification(list_of_lists, data, alpha):
  list_of_tuples = [tuple(inner_list) for inner_list in list_of_lists]
  i = 0
  while len(list_of_lists[i])>1:

      Z = linkage(data.x[list_of_tuples[i],:].detach().numpy(), method='average', metric='cosine')
      threshold = alpha * np.max(Z[:, 2])
      clusters = fcluster(Z, threshold, criterion='distance')

      # Get cluster labels for each data point
      labels = clusters


      # Z = linkage(data.x[list_of_tuples[i],:], method='average', metric='euclidean')
      # threshold = alpha * np.max(Z[:, 2])
      # clusters = fcluster(Z, threshold, criterion='distance')

      # # Get cluster labels for each data point
      # labels = clusters




      # Sample tuple to split
      data_tuple = list_of_tuples[i]

      # Create a dictionary to store split data
      split_data = {}

      # Iterate through the labels and tuple elements
      for label, value in zip(labels, data_tuple):
        if label not in split_data:
          split_data[label] = []
        split_data[label].append(value)

      # Convert the dictionary to a list of tuples (if needed)
      split_tuples = [tuple(values) for key, values in split_data.items()]
      list_of_tuples.pop(i)
      j = 0
      for values in split_tuples:
        if j == 0:
          list_of_tuples.insert(0, values)
        else:
          list_of_tuples.insert(len(list_of_tuples), values)
        j+=1

      i+=1

  result_dict = {element: index for index, tup in enumerate(list_of_tuples) for element in tup}

  return list_of_tuples, result_dict

In [ ]:
# def Sparisified_G_NX(list_of_tuples, result_dict):
#   BCG = nx.DiGraph()

#   for i in range(len(list_of_tuples)):
#     BCG.add_node(i)

#   for edge in G.edges():
#       source, target = edge
#       BCG.add_edge(result_dict[source], result_dict[target])

#   # Get the number of nodes and edges
#   num_nodes = BCG.number_of_nodes()
#   num_edges = BCG.number_of_edges()



#   print("Number of nodes:", num_nodes)
#   print("Number of edges:", num_edges)


#   return BCG

In [ ]:
def Sparisified_G_NX(list_of_tuples, result_dict, data):
  BCG = nx.DiGraph()

  for i in range(len(list_of_tuples)):
    BCG.add_node(i)

  for edge in G.edges():
      source, target = edge
      # if cosine_similarity(data.x[source].numpy().reshape(1, -1), data.x[target].numpy().reshape(1, -1)) > 0.5:
      BCG.add_edge(result_dict[source], result_dict[target])

  num_nodes = BCG.number_of_nodes()
  num_edges = BCG.number_of_edges()



  print("Number of nodes:", num_nodes)
  print("Number of edges:", num_edges)


  return BCG

In [ ]:
def Sparisified_G_Alpha(BCG, data, list_of_tuples):
  num_nodes = BCG.number_of_nodes()
  new_x = torch.zeros(num_nodes,data.x.shape[1])
  new_y = torch.zeros(num_nodes)
  for i in range(len(list_of_tuples)):
    new_x[i,:] = torch.mean(data.x[list(list_of_tuples[i]),:], axis = 0)
    EC = list_of_tuples[i]
    L = []
    for ele in EC:
      L.append(int(data.y[ele].numpy()))

    keys = list(Counter(L).keys())
    frequencies = list(Counter(L).values())

    # Randomly select a key based on frequencies
    random_key = random.choices(keys, frequencies)[0]
    new_y[i] = int(random_key)

  edges = list(BCG.edges())

  # Create a dictionary to map node labels to integer indices
  node_mapping = {node: index for index, node in enumerate(BCG.nodes())}

  # Convert the edges to indices using the node_mapping
  new_edge_index = [[node_mapping[source], node_mapping[target]] for source, target in edges]
  new_edge_index = torch.tensor(new_edge_index, dtype=torch.long).t().contiguous()

  data_new1 = copy.copy(data)
  data_new1.x = new_x
  data_new1.y = new_y.long()
  data_new1.edge_index = new_edge_index
  data_new1.num_nodes = len(new_y)



  # for u, v in BCG.edges():
  #   similarity = cosine_similarity(new_x[u].numpy().reshape(1, -1), new_x[v].numpy().reshape(1, -1))
  #   BCG[u][v]['weight'] = similarity[0][0]

  # edge_weights = [BCG[u][v]['weight'] for u, v in BCG.edges()]
  # # Convert edge weights to a PyTorch tensor
  # edge_weight_tensor = torch.tensor(edge_weights, dtype=torch.float32)

  return data_new1

In [ ]:
def Sparisified_G_Alpha_0(G, data):

  new_x = torch.zeros(len(list(G.nodes)),data.x.shape[1])
  new_y = torch.zeros(len(list(G.nodes)))
  list_itr = list(G.nodes)
  for i in range(len(list(G.nodes))):
    new_x[i,:] = data.x[list_itr[i],:]
    new_y[i] = int(data.y[list_itr[i]])

  edges = list(G.edges())

  # Create a dictionary to map node labels to integer indices
  node_mapping = {node: index for index, node in enumerate(G.nodes())}

  # Convert the edges to indices using the node_mapping
  new_edge_index = [[node_mapping[source], node_mapping[target]] for source, target in edges]
  new_edge_index = torch.tensor(new_edge_index, dtype=torch.long).t().contiguous()


  data_new1 = copy.copy(data)
  data_new1.x = new_x
  data_new1.y = new_y.long()
  data_new1.edge_index = new_edge_index
  data_new1.num_nodes = len(new_y)

  # for u, v in G.edges():
  #   similarity = cosine_similarity(data.x[u].numpy().reshape(1, -1), data.x[v].numpy().reshape(1, -1))
  #   G[u][v]['weight'] = similarity[0][0]

  # edge_weights = [G[u][v]['weight'] for u, v in G.edges()]
  # # Convert edge weights to a PyTorch tensor
  # edge_weight_tensor = torch.tensor(edge_weights, dtype=torch.float32)

  return data_new1

In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
# from torch_geometric.nn import GATConv, GCNConv, GraphSAGE, BatchNorm
# import torch

# class GCN(torch.nn.Module):
#   def __init__(self):
#     super().__init__()

#     self.conv1 = GCNConv(dataset.num_node_features, 128,aggr = "sum")
#     self.bc1 = BatchNorm(128)
#     self.conv2 = GCNConv(128, 128,aggr = 'mean')
#     self.bc2 = BatchNorm(128)
#     self.conv3 = GCNConv(128, dataset.num_classes,aggr = 'mean')
#     self.bc3 = BatchNorm(dataset.num_classes)
#   def forward(self, x, edge_index, edge_weights):
#     # x: Node feature matrix
#     # edge_index: Graph connectivity matrix
#     x = self.conv1(x, edge_index, edge_weights)
#     x = torch.nn.functional.relu(x)
#     x = self.bc1(x)
#     x = self.conv2(x, edge_index, edge_weights)
#     x = torch.nn.functional.relu(x)
#     x = self.bc2(x)
#     x = self.conv3(x, edge_index, edge_weights)
#     x = torch.nn.functional.relu(x)
#     x = self.bc3(x)
#     return torch.nn.functional.log_softmax(x, dim=1)

# model = GCN()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# print("Graph Convolutional Network (GCN):")
# GCN()

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, SAGEConv, BatchNorm
import torch
from torch.nn import Module, Dropout

class GCN(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = GCNConv(data.num_node_features, 256)
    self.conv2 = GCNConv(256, 256)
    self.conv3 = GCNConv(256, 47)
    self.dropout = Dropout(p=0.5)
  def forward(self, x, edge_index):
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.dropout(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.dropout(x)
    x = self.conv3(x, edge_index)
    x = torch.nn.functional.relu(x)

    return torch.nn.functional.log_softmax(x, dim=1)

model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)
print("Graph Convolutional Network (GCN):")
GCN()

Graph Convolutional Network (GCN):


GCN(
  (conv1): GCNConv(100, 256)
  (conv2): GCNConv(256, 256)
  (conv3): GCNConv(256, 47)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, SAGEConv, BatchNorm
import torch
from torch.nn import Module, Dropout
class GraphSAGE(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = SAGEConv(data.num_node_features, 256)
    self.conv2 = SAGEConv(256, 256)
    self.conv3 = SAGEConv(256, 47)
    self.dropout = Dropout(p=0.5)

  def forward(self, x, edge_index):
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.dropout(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.dropout(x)
    x = self.conv3(x, edge_index)
    x = torch.nn.functional.relu(x)

    return torch.nn.functional.log_softmax(x, dim=1)

model = GraphSAGE()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("GraphSAGE:")
GraphSAGE()

GraphSAGE:


GraphSAGE(
  (conv1): SAGEConv(100, 256, aggr=mean)
  (conv2): SAGEConv(256, 256, aggr=mean)
  (conv3): SAGEConv(256, 47, aggr=mean)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, SAGEConv, BatchNorm
import torch
from torch.nn import Module, Dropout

class GAT(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = GATConv(data.num_node_features, 256)
    self.conv2 = GATConv(256, 256)
    self.conv3 = GATConv(256, 47)
    self.dropout = Dropout(p=0.5)

  def forward(self, x, edge_index):
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.dropout(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.dropout(x)
    x = self.conv3(x, edge_index)
    x = torch.nn.functional.relu(x)

    return torch.nn.functional.log_softmax(x, dim=1)

model = GAT()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("GAT:")
GAT()

GAT:


GAT(
  (conv1): GATConv(100, 256, heads=1)
  (conv2): GATConv(256, 256, heads=1)
  (conv3): GATConv(256, 47, heads=1)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [ ]:
def compute_accuracy(pred_y, y):
    return (pred_y == y).sum()

In [ ]:
# def exclude_tested_elements_from_MB(partition, tested_elements):
#   tuple_list = partition

#   # Sort the list by the number of elements in each tuple
#   sorted_list = sorted(tuple_list, key=lambda x: len(x), reverse=True)

#   indices = []
#   list_of_lists = [list(t) for t in sorted_list]

#   for ele in tested_elements:
#     id = find_tuple_index_with_integer(sorted_list, ele)
#     if len(list_of_lists[id]) > 1:
#       list_of_lists[id].remove(ele)
#       list_of_lists.append([ele])
#       indices.append(len(list_of_lists)-1)
#     else:
#       indices.append(id)

#   return list_of_lists

# def alpha_Sparsification(list_of_lists, data, alpha):
#   list_of_tuples = [tuple(inner_list) for inner_list in list_of_lists]
#   i = 0
#   while len(list_of_lists[i])>1:
#       # print(i)

#       Z = linkage(data.x[list_of_tuples[i],:], method='average', metric='cosine')
#       threshold = alpha * np.max(Z[:, 2])
#       clusters = fcluster(Z, threshold, criterion='distance')

#       # Get cluster labels for each data point
#       labels = clusters

#       # Sample tuple to split
#       data_tuple = list_of_tuples[i]

#       # Create a dictionary to store split data
#       split_data = {}

#       # Iterate through the labels and tuple elements
#       for label, value in zip(labels, data_tuple):
#         if label not in split_data:
#           split_data[label] = []
#         split_data[label].append(value)

#       # Convert the dictionary to a list of tuples (if needed)
#       split_tuples = [tuple(values) for key, values in split_data.items()]
#       list_of_tuples.pop(i)
#       j = 0
#       for values in split_tuples:
#         if j == 0:
#           list_of_tuples.insert(0, values)
#         else:
#           list_of_tuples.insert(len(list_of_tuples), values)
#         j+=1

#       i+=1

#   result_dict = {element: index for index, tup in enumerate(list_of_tuples) for element in tup}

#   return list_of_tuples, result_dict, indices

In [ ]:
similarity = cosine_similarity(data.x[0].numpy().reshape(1, -1), data.x[100].numpy().reshape(1, -1))
similarity[0][0]

0.8133455

In [ ]:
for u, v in G.edges():
    similarity = cosine_similarity(data.x[u].numpy().reshape(1, -1), data.x[v].numpy().reshape(1, -1))
    G[u][v]['weight'] = similarity[0][0]

edge_weights = [G[u][v]['weight'] for u, v in G.edges()]
# Convert edge weights to a PyTorch tensor
edge_weight_tensor = torch.tensor(edge_weights, dtype=torch.float32)
edge_weight_tensor.shape

torch.Size([221911])

In [ ]:
# for edge_list in list(G.edges):
#   similarity = cosine_similarity(data.x[edge_list[0]].numpy().reshape(1, -1), data.x[edge_list[1]].numpy().reshape(1, -1))
#   G[edge_list[0]][edge_list[1]] = similarity

# edge_weights = [G[u][v]['weight'] for u, v in BCG.edges()]
# for u, v in G.edges():
#     similarity = cosine_similarity([node_features[u]], [node_features[v]])[0][0]
#     G[u][v]['weight'] = similarity


# # Convert edge weights to a PyTorch tensor
# edge_weight_tensor = torch.tensor(edge_weights, dtype=torch.float32)
# edge_weight_tensor

In [ ]:
model_save_path_GCN

'gdrive/My Drive/GNN_Sparsification/Pre_Trained_5%_ogbn_products_GCN.pt'

In [ ]:
# evaluate the model on test set
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

# best_model_GAT = GAT()
# best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))


alpha_list = np.arange(0, 1.1, 0.25)
num_of_runs = 10

AVG_time_GCN = 0
AVG_time_GAT = 0
AVG_time_GraphSAGE = 0

for i in range(num_of_runs):
  start_time = time.time()
  best_model_GCN.eval()
  pred = best_model_GCN(data.x, data.edge_index).argmax(dim=1)
  # correct = compute_accuracy(pred, test_data.y)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GCN += elapsed_time

  if i == num_of_runs-1:
    print("GCN Inference time on original G:", AVG_time_GCN/num_of_runs, "seconds")
    correct = compute_accuracy(pred[test_mask], data.y[test_mask])
    # acc = int(correct) / len(test_data.y)
    acc = int(correct) / len(tested_elements)
    print(f'GCN Inference Accuracy: {acc:.4f}')

# for j in range(num_of_runs):
#   start_time = time.time()
#   best_model_GAT.eval()
#   pred = best_model_GAT(data.x, data.edge_index).argmax(dim=1)
#   # correct = compute_accuracy(pred, test_data.y)

#   # End recording the current time
#   end_time = time.time()

#   # Calculate the elapsed time
#   elapsed_time = end_time - start_time
#   AVG_time_GAT += elapsed_time

#   if j == num_of_runs-1:
#     print("GAT Inference time on original G:", AVG_time_GAT/num_of_runs, "seconds")
#     correct = compute_accuracy(pred[test_mask], data.y[test_mask])
#     # acc = int(correct) / len(test_data.y)
#     acc = int(correct) / len(tested_elements)
#     print(f'GAT Inference Accuracy: {acc:.4f}')

for n in range(num_of_runs):
  start_time = time.time()
  best_model_GraphSage.eval()
  pred = best_model_GraphSage(data.x, data.edge_index).argmax(dim=1)
  # correct = compute_accuracy(pred, test_data.y)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GraphSAGE += elapsed_time

  if n == num_of_runs-1:
    print("GraphSAGE Inference time on original G:", AVG_time_GraphSAGE/num_of_runs, "seconds")
    correct = compute_accuracy(pred[test_mask], data.y[test_mask])
    # acc = int(correct) / len(test_data.y)
    acc = int(correct) / len(tested_elements)
    print(f'GraphSAGE Inference Accuracy: {acc:.4f}')


In [ ]:
data

Data(num_nodes=169343, edge_index=[2, 1166243], x=[169343, 128], node_year=[169343, 1], y=[169343])

In [ ]:
# # evaluate the model on test set
# best_model_GCN = GCN()
# best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

# best_model_GAT = GAT()
# best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

# best_model_GraphSage = GraphSAGE()
# best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))


# alpha_list = np.arange(0, 1.1, 0.25)
# num_of_runs = 50

# AVG_time_GCN = 0
# AVG_time_GAT = 0
# AVG_time_GraphSAGE = 0

# for i in range(num_of_runs):
#   start_time = time.time()
#   best_model_GCN.eval()
#   pred = best_model_GCN(data.x, data.edge_index).argmax(dim=1)
#   # correct = compute_accuracy(pred, test_data.y)

#   # End recording the current time
#   end_time = time.time()

#   # Calculate the elapsed time
#   elapsed_time = end_time - start_time
#   AVG_time_GCN += elapsed_time

#   if i == num_of_runs-1:
#     print("GCN Inference time on original G:", AVG_time_GCN/num_of_runs, "seconds")
#     correct = compute_accuracy(pred[test_mask], data.y[test_mask])
#     # acc = int(correct) / len(test_data.y)
#     acc = int(correct) / len(tested_elements)
#     print(f'GCN Inference Accuracy: {acc:.4f}')

# for j in range(num_of_runs):
#   start_time = time.time()
#   best_model_GAT.eval()
#   pred = best_model_GAT(data.x, data.edge_index).argmax(dim=1)
#   # correct = compute_accuracy(pred, test_data.y)

#   # End recording the current time
#   end_time = time.time()

#   # Calculate the elapsed time
#   elapsed_time = end_time - start_time
#   AVG_time_GAT += elapsed_time

#   if j == num_of_runs-1:
#     print("GAT Inference time on original G:", AVG_time_GAT/num_of_runs, "seconds")
#     correct = compute_accuracy(pred[test_mask], data.y[test_mask])
#     # acc = int(correct) / len(test_data.y)
#     acc = int(correct) / len(tested_elements)
#     print(f'GAT Inference Accuracy: {acc:.4f}')

# for n in range(num_of_runs):
#   start_time = time.time()
#   best_model_GraphSage.eval()
#   pred = best_model_GraphSage(data.x, data.edge_index).argmax(dim=1)
#   # correct = compute_accuracy(pred, test_data.y)

#   # End recording the current time
#   end_time = time.time()

#   # Calculate the elapsed time
#   elapsed_time = end_time - start_time
#   AVG_time_GraphSAGE += elapsed_time

#   if n == num_of_runs-1:
#     print("GraphSAGE Inference time on original G:", AVG_time_GraphSAGE/num_of_runs, "seconds")
#     correct = compute_accuracy(pred[test_mask], data.y[test_mask])
#     # acc = int(correct) / len(test_data.y)
#     acc = int(correct) / len(tested_elements)
#     print(f'GraphSAGE Inference Accuracy: {acc:.4f}')


In [ ]:
Constant_GCN = AVG_time_GCN/num_of_runs
Constant_GAT = AVG_time_GAT/num_of_runs
Constant_GraphSAGE = AVG_time_GraphSAGE/num_of_runs

NameError: name 'AVG_time_GCN' is not defined

In [ ]:
Constant_GCN = 1
Constant_GAT = 1
Constant_GraphSAGE = 1

In [ ]:
data_new_1.edge_index.shape

torch.Size([2, 221911])

# K-hop-Induced + SPGS

In [ ]:
# Generate the first piece with step size 0.01 for the range [0, 0.15)
array1 = np.arange(0, 0.01, 0.005)

# Generate the second piece with step size 0.02 for the range [0.15, 0.3)
array2 = np.arange(0.01, 0.2, 0.02)



# Concatenate the two arrays
alpha_list = np.concatenate([array1, array2])
# alpha_list = np.arange(0, 0.4, 0.05)

In [ ]:
tested_elements_set = set(tested_elements)
indices = [index for index, node in enumerate(G.nodes) if node in tested_elements_set]
len(indices)

91998

In [ ]:
# alpha_list = np.arange(0.5, 1.1, 0.5)
alpha_list = [0, 0.05, 1]
alpha_list

[0, 0.05, 1]

In [ ]:
list_of_lists, indices = exclude_tested_elements_from_MB(partition, tested_elements)

In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

# best_model_GAT = GAT()
# best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

# best_modetested_elementsl_GraphSage = GraphSAGE()
# best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))

# num_of_runs = 5

# alpha_list = np.arange(0.1, 0.5, 0.1)
for alpha in alpha_list:
  # print(alpha)
  if alpha == 0:
    start_time = time.time()
    data_new_1 = Sparisified_G_Alpha_0(G, data)
    end_time = time.time()
    execution_time_SPGS = end_time - start_time
    print(f"Execution time: {execution_time_SPGS} seconds")

    tested_elements_set = set(tested_elements)
    indices = [index for index, node in enumerate(G.nodes) if node in tested_elements_set]


  if alpha > 0 and alpha < 1:
    print(alpha)
    if alpha == alpha_list[0]:
      start_time_1 = time.time()
      list_of_lists, indices = exclude_tested_elements_from_MB(partition, tested_elements)


    list_of_tuples, result_dict = alpha_Sparsification(list_of_lists, data, alpha)
    # end_time_1 = time.time()
    # execution_time_1 = end_time_1 - start_time_1
    # print(f"Execution time of alpha_Sparsification: {execution_time_1} seconds")

    BCG = Sparisified_G_NX(list_of_tuples, result_dict, data)
    data_new_1 = Sparisified_G_Alpha(BCG, data, list_of_tuples)
    end_time_2 = time.time()
    execution_time_2 = end_time_2 - start_time_1
    print(f"Execution time of  (alpha, r) SPGC: {execution_time_2} seconds")

  if alpha == 1:
    l_o_t = [tuple(inner_list) for inner_list in list_of_lists]
    r_d = {element: index for index, tup in enumerate(list_of_tuples) for element in tup}
    start_time_3 = time.time()
    BCG = Sparisified_G_NX(l_o_t, r_d, data)
    data_new_1 = Sparisified_G_Alpha(BCG, data, l_o_t)
    end_time_3 = time.time()
    execution_time_3 = end_time_3 - start_time_3
    print(f"Execution time of SPGC: {execution_time_3} seconds")


  # AVG_time_GCN = 0
  # AVG_time_GAT = 0
  # AVG_time_GraphSAGE = 0

  # for i in range(num_of_runs):
  #   start_time = time.time()
  #   best_model_GCN.eval()
  #   pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

  #   # End recording the current time
  #   end_time = time.time()

  #   # Calculate the elapsed time
  #   elapsed_time = end_time - start_time
  #   AVG_time_GCN += elapsed_time

  #   if i == num_of_runs-1:
  #     correct = compute_accuracy(pred[indices], data_new_1.y[indices])
  #     acc = int(correct) / len(tested_elements)
  #     # correct = compute_accuracy(pred, data_new_1.y)
  #     # acc = int(correct) / len(pred)
  #     print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
  #     print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
  #     print(f'GCN Inference Accuracy: {acc:.4f}')

  # for j in range(num_of_runs):
  #   AVG_time = 0
  #   start_time = time.time()
  #   best_model_GAT.eval()
  #   pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

  #   # End recording the current time
  #   end_time = time.time()

  #   # Calculate the elapsed time
  #   elapsed_time = end_time - start_time
  #   AVG_time_GAT += elapsed_time


  #   if j == num_of_runs-1:
  #     correct = compute_accuracy(pred[indices], data_new_1.y[indices])
  #     acc = int(correct) / len(tested_elements)
  #     print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
  #     print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
  #     print(f'GAT Inference Accuracy: {acc:.4f}')

  # for m in range(num_of_runs):
  #   start_time = time.time()
  #   best_model_GraphSage.eval()
  #   pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

  #   # End recording the current time
  #   end_time = time.time()

  #   # Calculate the elapsed time
  #   elapsed_time = end_time - start_time
  #   AVG_time_GraphSAGE += elapsed_time

  #   if m == num_of_runs-1:
  #     correct = compute_accuracy(pred[indices], data_new_1.y[indices])
  #     acc = int(correct) / len(tested_elements)
  #     print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
  #     print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
  #     print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

Execution time: 81.54737663269043 seconds
0.05
Number of nodes: 94032
Number of edges: 1196012
Execution time of  (alpha, r) SPGC: 1074.1715507507324 seconds
Number of nodes: 94032
Number of edges: 1196012
Execution time of SPGC: 46.07778072357178 seconds


In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

# best_model_GAT = GAT()
# best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

# best_modetested_elementsl_GraphSage = GraphSAGE()
# best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))

# num_of_runs = 5

# alpha_list = np.arange(0.1, 0.5, 0.1)
for alpha in alpha_list:
  # print(alpha)
  if alpha == 0:
    start_time = time.time()
    data_new_1 = Sparisified_G_Alpha_0(G, data)
    end_time = time.time()
    execution_time_SPGS = end_time - start_time
    print(f"Execution time: {execution_time_SPGS} seconds")

    tested_elements_set = set(tested_elements)
    indices = [index for index, node in enumerate(G.nodes) if node in tested_elements_set]


  if alpha > 0 and alpha < 1:
    print(alpha)
    if alpha == alpha_list[0]:
      start_time_1 = time.time()
      list_of_lists, indices = exclude_tested_elements_from_MB(partition, tested_elements)


    list_of_tuples, result_dict = alpha_Sparsification(list_of_lists, data, alpha)
    # end_time_1 = time.time()
    # execution_time_1 = end_time_1 - start_time_1
    # print(f"Execution time of alpha_Sparsification: {execution_time_1} seconds")

    BCG = Sparisified_G_NX(list_of_tuples, result_dict, data)
    data_new_1 = Sparisified_G_Alpha(BCG, data, list_of_tuples)
    end_time_2 = time.time()
    execution_time_2 = end_time_2 - start_time_1
    print(f"Execution time of  (alpha, r) SPGC: {execution_time_2} seconds")

  if alpha == 1:
    l_o_t = [tuple(inner_list) for inner_list in list_of_lists]
    r_d = {element: index for index, tup in enumerate(list_of_tuples) for element in tup}
    start_time_3 = time.time()
    BCG = Sparisified_G_NX(l_o_t, r_d, data)
    data_new_1 = Sparisified_G_Alpha(BCG, data, l_o_t)
    end_time_3 = time.time()
    execution_time_3 = end_time_3 - start_time_3
    print(f"Execution time of SPGC: {execution_time_3} seconds")


  # AVG_time_GCN = 0
  # AVG_time_GAT = 0
  # AVG_time_GraphSAGE = 0

  # for i in range(num_of_runs):
  #   start_time = time.time()
  #   best_model_GCN.eval()
  #   pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

  #   # End recording the current time
  #   end_time = time.time()

  #   # Calculate the elapsed time
  #   elapsed_time = end_time - start_time
  #   AVG_time_GCN += elapsed_time

  #   if i == num_of_runs-1:
  #     correct = compute_accuracy(pred[indices], data_new_1.y[indices])
  #     acc = int(correct) / len(tested_elements)
  #     # correct = compute_accuracy(pred, data_new_1.y)
  #     # acc = int(correct) / len(pred)
  #     print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
  #     print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
  #     print(f'GCN Inference Accuracy: {acc:.4f}')

  # for j in range(num_of_runs):
  #   AVG_time = 0
  #   start_time = time.time()
  #   best_model_GAT.eval()
  #   pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

  #   # End recording the current time
  #   end_time = time.time()

  #   # Calculate the elapsed time
  #   elapsed_time = end_time - start_time
  #   AVG_time_GAT += elapsed_time


  #   if j == num_of_runs-1:
  #     correct = compute_accuracy(pred[indices], data_new_1.y[indices])
  #     acc = int(correct) / len(tested_elements)
  #     print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
  #     print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
  #     print(f'GAT Inference Accuracy: {acc:.4f}')

  # for m in range(num_of_runs):
  #   start_time = time.time()
  #   best_model_GraphSage.eval()
  #   pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

  #   # End recording the current time
  #   end_time = time.time()

  #   # Calculate the elapsed time
  #   elapsed_time = end_time - start_time
  #   AVG_time_GraphSAGE += elapsed_time

  #   if m == num_of_runs-1:
  #     correct = compute_accuracy(pred[indices], data_new_1.y[indices])
  #     acc = int(correct) / len(tested_elements)
  #     print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
  #     print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
  #     print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

0.05
Number of nodes: 94032
Number of edges: 1196012
Execution time of  (alpha, r) SPGC: 47.45617961883545 seconds
Number of nodes: 94032
Number of edges: 1196012
Execution time of SPGC: 24.013351678848267 seconds


In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

# best_model_GAT = GAT()
# best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

# best_modetested_elementsl_GraphSage = GraphSAGE()
# best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))

# num_of_runs = 5

# alpha_list = np.arange(0.1, 0.5, 0.1)
for alpha in alpha_list:
  print(alpha)
  if alpha == 0:
    start_time = time.time()
    data_new_1 = Sparisified_G_Alpha_0(G, data)
    end_time = time.time()
    execution_time_SPGS = end_time - start_time
    print(f"Execution time: {execution_time_SPGS} seconds")

    tested_elements_set = set(tested_elements)
    indices = [index for index, node in enumerate(G.nodes) if node in tested_elements_set]


  if alpha > 0 and alpha < 1:
    if alpha == alpha_list[0]:
      list_of_lists, indices = exclude_tested_elements_from_MB(partition, tested_elements)

    start_time = time.time()
    list_of_tuples, result_dict = alpha_Sparsification(list_of_lists, data, alpha)
    BCG = Sparisified_G_NX(list_of_tuples, result_dict, data)
    data_new_1 = Sparisified_G_Alpha(BCG, data, list_of_tuples)
    end_time = time.time()
    execution_time_alpha_r_SPGS = end_time - start_time
    print(f"Execution time of (alpha, r) SPGC: {execution_time_alpha_r_SPGS} seconds")

  if alpha == 1:
    list_of_tuples = [tuple(inner_list) for inner_list in list_of_lists]
    result_dict = {element: index for index, tup in enumerate(list_of_tuples) for element in tup}
    start_time = time.time()
    BCG = Sparisified_G_NX(list_of_tuples, result_dict, data)
    data_new_1 = Sparisified_G_Alpha(BCG, data, list_of_tuples)
    end_time = time.time()
    execution_time_alpha_r_SPGS = end_time - start_time
    print(f"Execution time of SPGC: {execution_time_alpha_r_SPGS} seconds")


  # AVG_time_GCN = 0
  # AVG_time_GAT = 0
  # AVG_time_GraphSAGE = 0

  # for i in range(num_of_runs):
  #   start_time = time.time()
  #   best_model_GCN.eval()
  #   pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

  #   # End recording the current time
  #   end_time = time.time()

  #   # Calculate the elapsed time
  #   elapsed_time = end_time - start_time
  #   AVG_time_GCN += elapsed_time

  #   if i == num_of_runs-1:
  #     correct = compute_accuracy(pred[indices], data_new_1.y[indices])
  #     acc = int(correct) / len(tested_elements)
  #     # correct = compute_accuracy(pred, data_new_1.y)
  #     # acc = int(correct) / len(pred)
  #     print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
  #     print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
  #     print(f'GCN Inference Accuracy: {acc:.4f}')

  # for j in range(num_of_runs):
  #   AVG_time = 0
  #   start_time = time.time()
  #   best_model_GAT.eval()
  #   pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

  #   # End recording the current time
  #   end_time = time.time()

  #   # Calculate the elapsed time
  #   elapsed_time = end_time - start_time
  #   AVG_time_GAT += elapsed_time


  #   if j == num_of_runs-1:
  #     correct = compute_accuracy(pred[indices], data_new_1.y[indices])
  #     acc = int(correct) / len(tested_elements)
  #     print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
  #     print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
  #     print(f'GAT Inference Accuracy: {acc:.4f}')

  # for m in range(num_of_runs):
  #   start_time = time.time()
  #   best_model_GraphSage.eval()
  #   pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

  #   # End recording the current time
  #   end_time = time.time()

  #   # Calculate the elapsed time
  #   elapsed_time = end_time - start_time
  #   AVG_time_GraphSAGE += elapsed_time

  #   if m == num_of_runs-1:
  #     correct = compute_accuracy(pred[indices], data_new_1.y[indices])
  #     acc = int(correct) / len(tested_elements)
  #     print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
  #     print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
  #     print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

1.0
Number of nodes: 94032
Number of edges: 1196012
Execution time of SPGC: 45.592939138412476 seconds


In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))

num_of_runs = 100

# alpha_list = np.arange(0, 0.15, 0.01)

for alpha in alpha_list:
  print(alpha)
  if alpha == 0:
    data_new_1 = Sparisified_G_Alpha_0(G, data)
    list2 = tested_elements
    list1 = list(G.nodes)
    indices = [list1.index(x) for x in list2]

  if alpha > 0 and alpha < 1:
    if alpha == alpha_list[1]:
      list_of_lists, indices = exclude_tested_elements_from_MB(partition, tested_elements)

    list_of_tuples, result_dict = alpha_Sparsification(list_of_lists, data, alpha)
    BCG = Sparisified_G_NX(list_of_tuples, result_dict, data)
    data_new_1 = Sparisified_G_Alpha(BCG, data, list_of_tuples)

  if alpha = 1:
    if alpha == alpha_list[1]:
      list_of_lists, indices = exclude_tested_elements_from_MB(partition, tested_elements)

    list_of_tuples, result_dict = alpha_Sparsification(list_of_lists, data, alpha)
    BCG = Sparisified_G_NX(list_of_tuples, result_dict, data)
    data_new_1 = Sparisified_G_Alpha(BCG, data, list_of_tuples)

  AVG_time_GCN = 0
  AVG_time_GAT = 0
  AVG_time_GraphSAGE = 0

  for i in range(num_of_runs):
    start_time = time.time()
    best_model_GCN.eval()
    pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GCN += elapsed_time


    if i == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      # correct = compute_accuracy(pred, data_new_1.y)
      # acc = int(correct) / len(pred)
      print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
      print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
      print(f'GCN Inference Accuracy: {acc:.4f}')

  for j in range(num_of_runs):
    AVG_time = 0
    start_time = time.time()
    best_model_GAT.eval()
    pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GAT += elapsed_time


    if j == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
      print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
      print(f'GAT Inference Accuracy: {acc:.4f}')

  for m in range(num_of_runs):
    start_time = time.time()
    best_model_GraphSage.eval()
    pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GraphSAGE += elapsed_time



    if m == num_of_runs-1:
      correct = compute_accuracy(pred[indices], data_new_1.y[indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
      print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
      print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

0.0
Average of 100 runs GCN Inference time: 0.13984939098358154 seconds
GCN Inference Speed-up: 8.6476
GCN Inference Accuracy: 0.6016
Average of 100 runs GAT Inference time: 0.16920061826705932 seconds
GAT Inference Speed-up: 7.8971
GAT Inference Accuracy: 0.5971
Average of 100 runs GraphSAGE Inference time: 0.13215621709823608 seconds
GraphSAGE Inference Speed-up: 8.6381
GraphSAGE Inference Accuracy: 0.5869
0.005
Number of nodes: 46658
Number of edges: 86190
Average of 100 runs GCN Inference time: 0.07239089488983154 seconds
GCN Inference Speed-up: 16.7059
GCN Inference Accuracy: 0.5916
Average of 100 runs GAT Inference time: 0.1351801013946533 seconds
GAT Inference Speed-up: 9.8845
GAT Inference Accuracy: 0.5754
Average of 100 runs GraphSAGE Inference time: 0.1008339524269104 seconds
GraphSAGE Inference Speed-up: 11.3214
GraphSAGE Inference Accuracy: 0.5748
0.01
Number of nodes: 46651
Number of edges: 86184
Average of 100 runs GCN Inference time: 0.07816484928131104 seconds
GCN Infer

In [ ]:
# best_model_GCN = GCN()
# best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

# best_model_GAT = GAT()
# best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

# best_model_GraphSage = GraphSAGE()
# best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))


# alpha_list = np.arange(0, 1.1, 0.1)
# for alpha in alpha_list:
#   print(alpha)
#   if alpha == 0:
#     data_new_1 = Sparisified_G_Alpha_0(G, data)
#     list2 = tested_elements
#     list1 = list(G.nodes)
#     indices = [list1.index(x) for x in list2]

#   if alpha > 0:
#     if alpha == alpha_list[1]:
#       list_of_lists, indices = exclude_tested_elements_from_MB(partition, tested_elements)
#     list_of_tuples, result_dict = alpha_Sparsification(list_of_lists, data, alpha)
#     BCG = Sparisified_G_NX(list_of_tuples, result_dict, data)
#     data_new_1 = Sparisified_G_Alpha(BCG, data, list_of_tuples)

#   AVG_time_GCN = 0
#   AVG_time_GAT = 0
#   AVG_time_GraphSAGE = 0

#   for i in range(num_of_runs):
#     start_time = time.time()
#     best_model_GCN.eval()
#     pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

#     # End recording the current time
#     end_time = time.time()

#     # Calculate the elapsed time
#     elapsed_time = end_time - start_time
#     AVG_time_GCN += elapsed_time


#     if i == num_of_runs-1:
#       correct = compute_accuracy(pred[indices], data_new_1.y[indices])
#       acc = int(correct) / len(tested_elements)
#       # correct = compute_accuracy(pred, data_new_1.y)
#       # acc = int(correct) / len(pred)
#       print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
#       print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
#       print(f'GCN Inference Accuracy: {acc:.4f}')

#   for j in range(num_of_runs):
#     AVG_time = 0
#     start_time = time.time()
#     best_model_GAT.eval()
#     pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

#     # End recording the current time
#     end_time = time.time()

#     # Calculate the elapsed time
#     elapsed_time = end_time - start_time
#     AVG_time_GAT += elapsed_time


#     if j == num_of_runs-1:
#       correct = compute_accuracy(pred[indices], data_new_1.y[indices])
#       acc = int(correct) / len(tested_elements)
#       print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
#       print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
#       print(f'GAT Inference Accuracy: {acc:.4f}')

#   for m in range(num_of_runs):
#     start_time = time.time()
#     best_model_GraphSage.eval()
#     pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

#     # End recording the current time
#     end_time = time.time()

#     # Calculate the elapsed time
#     elapsed_time = end_time - start_time
#     AVG_time_GraphSAGE += elapsed_time



#     if m == num_of_runs-1:
#       correct = compute_accuracy(pred[indices], data_new_1.y[indices])
#       acc = int(correct) / len(tested_elements)
#       print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
#       print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
#       print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

0.0
Average of 50 runs GCN Inference time: 0.4301473188400269 seconds
GCN Inference Speed-up: 3.0013
GCN Inference Accuracy: 0.5712
Average of 50 runs GAT Inference time: 0.505151572227478 seconds
GAT Inference Speed-up: 2.8089
GAT Inference Accuracy: 0.5692
Average of 50 runs GraphSAGE Inference time: 0.25812444686889646 seconds
GraphSAGE Inference Speed-up: 3.0834
GraphSAGE Inference Accuracy: 0.5834
0.1
Number of nodes: 75704
Number of edges: 194968
Average of 50 runs GCN Inference time: 0.3393095874786377 seconds
GCN Inference Speed-up: 3.8048
GCN Inference Accuracy: 0.5609
Average of 50 runs GAT Inference time: 0.40030022621154787 seconds
GAT Inference Speed-up: 3.5446
GAT Inference Accuracy: 0.5641
Average of 50 runs GraphSAGE Inference time: 0.20231366634368897 seconds
GraphSAGE Inference Speed-up: 3.9340
GraphSAGE Inference Accuracy: 0.5797
0.2
Number of nodes: 73753
Number of edges: 194096
Average of 50 runs GCN Inference time: 0.3475301170349121 seconds
GCN Inference Speed-up

# K-hop-Induced + K-spanner

In [ ]:
new_x.shape

torch.Size([94773, 128])

In [ ]:
# evaluate the model on test set
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))

from collections import Counter
import random
import copy

new_x = torch.zeros(len(list(G.nodes)),data.x.shape[1])
new_y = torch.zeros(len(list(G.nodes)))

for i in range(len(list(G.nodes))):
  new_x[i,:] = data.x[mapping[i],:]
  new_y[i] = int(data.y[mapping[i]])

data_new_1 = copy.copy(data)
data_new_1.x = new_x
data_new_1.y = new_y.long()
data_new_1.num_nodes = len(new_y)

# loaded_tensor = torch.load('gdrive/My Drive/GNN_Sparsification/Baselines/K_spanner_INDUCED/10%/2_spanner.pt')
# loaded_tensor = torch.load('gdrive/My Drive/GNN_Sparsification/Baselines/K_spanner_INDUCED/10%/100_spanner.pt')
# loaded_tensor = torch.load('gdrive/My Drive/GNN_Sparsification/Baselines/Dspar_INDUCED/10%/10%_induced_arxiv_sparsified_0.65.pt')
# loaded_tensor = torch.load('gdrive/My Drive/GNN_Sparsification/Baselines/Dspar_INDUCED/10%/10%_dir_induced_arxiv_sparsified_0.25.pt')
loaded_tensor = torch.load('gdrive/My Drive/GNN_Sparsification/Baselines/Dspar_INDUCED/10%/10%_dir_induced_arxiv_sparsified_1.55.pt')
data_new_1.edge_index = loaded_tensor[1]
# data_new_1.edge_index = loaded_tensor
data_new_1

Data(num_nodes=94773, edge_index=[2, 53602], x=[94773, 128], node_year=[169343, 1], y=[94773])

In [ ]:
loaded_tensor = torch.load('gdrive/My Drive/GNN_Sparsification/Baselines/Dspar_INDUCED/10%/10%_dir_induced_arxiv_sparsified_0.25.pt')
loaded_tensor[1].shape

torch.Size([2, 115055])

In [ ]:
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

Number of nodes: 60907
Number of edges: 104187


In [ ]:
data_new_1

Data(num_nodes=94773, edge_index=[2, 230110], x=[94773, 128], node_year=[169343, 1], y=[94773])

In [ ]:
data_new_1

Data(num_nodes=94773, edge_index=[2, 208418], x=[94773, 128], node_year=[169343, 1], y=[94773])

In [ ]:
data

Data(num_nodes=169343, edge_index=[2, 1166243], x=[169343, 128], node_year=[169343, 1], y=[169343])

In [ ]:
len(tested_elements)

8427

In [ ]:
len(new_indices)

8427

In [ ]:
# Generate a list of keys where values belong to specified_values
new_indices = [key for key, value in mapping.items() if value in tested_elements]

# Print the result
print("Keys with specified values:", new_indices)

Keys with specified values: [0, 7, 10, 15, 17, 37, 39, 44, 47, 49, 54, 67, 70, 74, 81, 96, 101, 103, 107, 109, 113, 117, 123, 129, 133, 136, 153, 156, 158, 166, 172, 193, 198, 219, 228, 232, 233, 234, 237, 246, 256, 260, 266, 281, 286, 291, 294, 295, 297, 301, 302, 306, 308, 315, 323, 326, 340, 352, 355, 380, 392, 419, 422, 427, 437, 441, 443, 447, 451, 461, 476, 488, 497, 498, 500, 503, 506, 509, 515, 522, 535, 538, 542, 545, 554, 556, 562, 564, 567, 569, 570, 584, 590, 597, 601, 610, 615, 622, 631, 633, 660, 665, 682, 692, 696, 719, 722, 743, 744, 756, 764, 782, 791, 808, 811, 816, 818, 832, 837, 845, 846, 853, 857, 859, 862, 875, 878, 879, 880, 883, 893, 894, 896, 911, 913, 929, 949, 955, 957, 962, 965, 973, 974, 986, 993, 995, 1006, 1014, 1015, 1018, 1021, 1028, 1040, 1048, 1050, 1053, 1060, 1071, 1072, 1075, 1081, 1083, 1086, 1091, 1094, 1097, 1099, 1111, 1117, 1125, 1131, 1136, 1138, 1141, 1157, 1158, 1159, 1162, 1170, 1178, 1185, 1192, 1195, 1202, 1213, 1217, 1219, 1227, 1229, 1

In [ ]:
num_of_runs = 50

AVG_time_GCN = 0
AVG_time_GAT = 0
AVG_time_GraphSAGE = 0

for i in range(num_of_runs):
  start_time = time.time()
  best_model_GCN.eval()
  pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GCN += elapsed_time


  if i == num_of_runs-1:
    correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
    acc = int(correct) / len(tested_elements)
    print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
    print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
    print(f'GCN Inference Accuracy: {acc:.4f}')

for j in range(num_of_runs):
  AVG_time = 0
  start_time = time.time()
  best_model_GAT.eval()
  pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GAT += elapsed_time


  if j == num_of_runs-1:
    correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
    acc = int(correct) / len(tested_elements)
    print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
    print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
    print(f'GAT Inference Accuracy: {acc:.4f}')

for m in range(num_of_runs):
  start_time = time.time()
  best_model_GraphSage.eval()
  pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GraphSAGE += elapsed_time



  if m == num_of_runs-1:
    correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
    acc = int(correct) / len(tested_elements)
    print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
    print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
    print(f'GraphSAGE Inference Accuracy: {acc:.4f}')


Average of 50 runs GCN Inference time: 0.39056416988372805 seconds
GCN Inference Speed-up: 3.3539
GCN Inference Accuracy: 0.3208
Average of 50 runs GAT Inference time: 0.4396880054473877 seconds
GAT Inference Speed-up: 3.5669
GAT Inference Accuracy: 0.3473
Average of 50 runs GraphSAGE Inference time: 0.21559114933013915 seconds
GraphSAGE Inference Speed-up: 3.7307
GraphSAGE Inference Accuracy: 0.4518


In [ ]:
num_of_runs = 50

AVG_time_GCN = 0
AVG_time_GAT = 0
AVG_time_GraphSAGE = 0

for i in range(num_of_runs):
  start_time = time.time()
  best_model_GCN.eval()
  pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GCN += elapsed_time


  if i == num_of_runs-1:
    correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
    acc = int(correct) / len(tested_elements)
    print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
    print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
    print(f'GCN Inference Accuracy: {acc:.4f}')

for j in range(num_of_runs):
  AVG_time = 0
  start_time = time.time()
  best_model_GAT.eval()
  pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GAT += elapsed_time


  if j == num_of_runs-1:
    correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
    acc = int(correct) / len(tested_elements)
    print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
    print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
    print(f'GAT Inference Accuracy: {acc:.4f}')

for m in range(num_of_runs):
  start_time = time.time()
  best_model_GraphSage.eval()
  pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GraphSAGE += elapsed_time



  if m == num_of_runs-1:
    correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
    acc = int(correct) / len(tested_elements)
    print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
    print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
    print(f'GraphSAGE Inference Accuracy: {acc:.4f}')


Average of 50 runs GCN Inference time: 0.3529387378692627 seconds
GCN Inference Speed-up: 3.7114
GCN Inference Accuracy: 0.3209
Average of 50 runs GAT Inference time: 0.4436634969711304 seconds
GAT Inference Speed-up: 3.5350
GAT Inference Accuracy: 0.3472
Average of 50 runs GraphSAGE Inference time: 0.21452019691467286 seconds
GraphSAGE Inference Speed-up: 3.7493
GraphSAGE Inference Accuracy: 0.4520


# K-hop-Induced + Dspar

In [ ]:
num_of_runs = 50

AVG_time_GCN = 0
AVG_time_GAT = 0
AVG_time_GraphSAGE = 0

for i in range(num_of_runs):
  start_time = time.time()
  best_model_GCN.eval()
  pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GCN += elapsed_time


  if i == num_of_runs-1:
    correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
    acc = int(correct) / len(tested_elements)
    print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
    print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
    print(f'GCN Inference Accuracy: {acc:.4f}')

for j in range(num_of_runs):
  AVG_time = 0
  start_time = time.time()
  best_model_GAT.eval()
  pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GAT += elapsed_time


  if j == num_of_runs-1:
    correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
    acc = int(correct) / len(tested_elements)
    print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
    print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
    print(f'GAT Inference Accuracy: {acc:.4f}')

for m in range(num_of_runs):
  start_time = time.time()
  best_model_GraphSage.eval()
  pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GraphSAGE += elapsed_time



  if m == num_of_runs-1:
    correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
    acc = int(correct) / len(tested_elements)
    print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
    print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
    print(f'GraphSAGE Inference Accuracy: {acc:.4f}')


Average of 50 runs GCN Inference time: 0.2660360050201416 seconds
GCN Inference Speed-up: 3.5297
GCN Inference Accuracy: 0.5631
Average of 50 runs GAT Inference time: 0.32143678665161135 seconds
GAT Inference Speed-up: 3.4051
GAT Inference Accuracy: 0.5623
Average of 50 runs GraphSAGE Inference time: 0.17017542839050293 seconds
GraphSAGE Inference Speed-up: 3.5158
GraphSAGE Inference Accuracy: 0.5751


In [ ]:
num_of_runs = 50

AVG_time_GCN = 0
AVG_time_GAT = 0
AVG_time_GraphSAGE = 0

for i in range(num_of_runs):
  start_time = time.time()
  best_model_GCN.eval()
  pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GCN += elapsed_time


  if i == num_of_runs-1:
    correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
    acc = int(correct) / len(tested_elements)
    print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
    print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
    print(f'GCN Inference Accuracy: {acc:.4f}')

for j in range(num_of_runs):
  AVG_time = 0
  start_time = time.time()
  best_model_GAT.eval()
  pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GAT += elapsed_time


  if j == num_of_runs-1:
    correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
    acc = int(correct) / len(tested_elements)
    print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
    print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
    print(f'GAT Inference Accuracy: {acc:.4f}')

for m in range(num_of_runs):
  start_time = time.time()
  best_model_GraphSage.eval()
  pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

  # End recording the current time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time
  AVG_time_GraphSAGE += elapsed_time



  if m == num_of_runs-1:
    correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
    acc = int(correct) / len(tested_elements)
    print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
    print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
    print(f'GraphSAGE Inference Accuracy: {acc:.4f}')


Average of 50 runs GCN Inference time: 0.21798655033111572 seconds
GCN Inference Speed-up: 4.3078
GCN Inference Accuracy: 0.5390
Average of 50 runs GAT Inference time: 0.2701277208328247 seconds
GAT Inference Speed-up: 4.0519
GAT Inference Accuracy: 0.5484
Average of 50 runs GraphSAGE Inference time: 0.14477694511413575 seconds
GraphSAGE Inference Speed-up: 4.1326
GraphSAGE Inference Accuracy: 0.5565


In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))
num_of_runs = 100


epsilon = 0.35
while epsilon <= 1.55:
  new_x = torch.zeros(len(list(G.nodes)),data.x.shape[1])
  new_y = torch.zeros(len(list(G.nodes)))

  for i in range(len(list(G.nodes))):
    new_x[i,:] = data.x[mapping[i],:]
    new_y[i] = int(data.y[mapping[i]])

  data_new_1 = copy.copy(data)
  data_new_1.x = new_x
  data_new_1.y = new_y.long()
  data_new_1.num_nodes = len(new_y)
  loaded_tensor = torch.load(f"gdrive/My Drive/GNN_Sparsification/Baselines/Dspar_INDUCED/15%/15%_dir_induced_arxiv_sparsified_{epsilon:.2f}.pt")
  data_new_1.edge_index = loaded_tensor[1]

  print(epsilon)


  AVG_time_GCN = 0
  AVG_time_GAT = 0
  AVG_time_GraphSAGE = 0

  for i in range(num_of_runs):
    start_time = time.time()
    best_model_GCN.eval()
    pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GCN += elapsed_time


    if i == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
      print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
      print(f'GCN Inference Accuracy: {acc:.4f}')

  for j in range(num_of_runs):
    AVG_time = 0
    start_time = time.time()
    best_model_GAT.eval()
    pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GAT += elapsed_time


    if j == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
      print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
      print(f'GAT Inference Accuracy: {acc:.4f}')

  for m in range(num_of_runs):
    start_time = time.time()
    best_model_GraphSage.eval()
    pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GraphSAGE += elapsed_time



    if m == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
      print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
      print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

  epsilon+=0.2


0.35
Average of 100 runs GCN Inference time: 0.4953016328811646 seconds
GCN Inference Speed-up: 2.5067
GCN Inference Accuracy: 0.5813
Average of 100 runs GAT Inference time: 0.587118182182312 seconds
GAT Inference Speed-up: 2.3032
GAT Inference Accuracy: 0.5866
Average of 100 runs GraphSAGE Inference time: 0.5065765285491943 seconds
GraphSAGE Inference Speed-up: 2.3690
GraphSAGE Inference Accuracy: 0.5677
0.55
Average of 100 runs GCN Inference time: 0.47952392101287844 seconds
GCN Inference Speed-up: 2.5892
GCN Inference Accuracy: 0.5809
Average of 100 runs GAT Inference time: 0.6021181416511535 seconds
GAT Inference Speed-up: 2.2458
GAT Inference Accuracy: 0.5861
Average of 100 runs GraphSAGE Inference time: 0.5730314588546753 seconds
GraphSAGE Inference Speed-up: 2.0943
GraphSAGE Inference Accuracy: 0.5677
0.75
Average of 100 runs GCN Inference time: 0.5036098694801331 seconds
GCN Inference Speed-up: 2.4653
GCN Inference Accuracy: 0.5766
Average of 100 runs GAT Inference time: 0.5915

In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))
num_of_runs = 100


epsilon = 0.35
while epsilon <= 1.55:
  new_x = torch.zeros(len(list(G.nodes)),data.x.shape[1])
  new_y = torch.zeros(len(list(G.nodes)))

  for i in range(len(list(G.nodes))):
    new_x[i,:] = data.x[mapping[i],:]
    new_y[i] = int(data.y[mapping[i]])

  data_new_1 = copy.copy(data)
  data_new_1.x = new_x
  data_new_1.y = new_y.long()
  data_new_1.num_nodes = len(new_y)
  loaded_tensor = torch.load(f"gdrive/My Drive/GNN_Sparsification/Baselines/Dspar_INDUCED/5%/5%_dir_induced_arxiv_sparsified_{epsilon:.2f}.pt")
  data_new_1.edge_index = loaded_tensor[1]

  print(epsilon)


  AVG_time_GCN = 0
  AVG_time_GAT = 0
  AVG_time_GraphSAGE = 0

  for i in range(num_of_runs):
    start_time = time.time()
    best_model_GCN.eval()
    pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GCN += elapsed_time


    if i == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
      print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
      print(f'GCN Inference Accuracy: {acc:.4f}')

  for j in range(num_of_runs):
    AVG_time = 0
    start_time = time.time()
    best_model_GAT.eval()
    pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GAT += elapsed_time


    if j == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
      print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
      print(f'GAT Inference Accuracy: {acc:.4f}')

  for m in range(num_of_runs):
    start_time = time.time()
    best_model_GraphSage.eval()
    pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GraphSAGE += elapsed_time



    if m == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
      print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
      print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

  epsilon+=0.2


0.35
Average of 100 runs GCN Inference time: 0.1054825735092163 seconds
GCN Inference Speed-up: 11.6881
GCN Inference Accuracy: 0.5913
Average of 100 runs GAT Inference time: 0.15996865034103394 seconds
GAT Inference Speed-up: 8.9148
GAT Inference Accuracy: 0.5933
Average of 100 runs GraphSAGE Inference time: 0.12928671836853028 seconds
GraphSAGE Inference Speed-up: 10.0053
GraphSAGE Inference Accuracy: 0.5873
0.55
Average of 100 runs GCN Inference time: 0.13353700160980225 seconds
GCN Inference Speed-up: 9.2326
GCN Inference Accuracy: 0.5910
Average of 100 runs GAT Inference time: 0.15879172801971436 seconds
GAT Inference Speed-up: 8.9808
GAT Inference Accuracy: 0.5931
Average of 100 runs GraphSAGE Inference time: 0.10846877813339234 seconds
GraphSAGE Inference Speed-up: 11.9256
GraphSAGE Inference Accuracy: 0.5870
0.75
Average of 100 runs GCN Inference time: 0.11737947702407837 seconds
GCN Inference Speed-up: 10.5034
GCN Inference Accuracy: 0.5887
Average of 100 runs GAT Inference ti

In [ ]:
best_model_GCN = GCN()
best_model_GCN.load_state_dict(torch.load(model_save_path_GCN))

best_model_GAT = GAT()
best_model_GAT.load_state_dict(torch.load(model_save_path_GAT))

best_model_GraphSage = GraphSAGE()
best_model_GraphSage.load_state_dict(torch.load(model_save_path_GraphSage))
num_of_runs = 100


epsilon = 0.35
while epsilon <= 1.55:
  new_x = torch.zeros(len(list(G.nodes)),data.x.shape[1])
  new_y = torch.zeros(len(list(G.nodes)))

  for i in range(len(list(G.nodes))):
    new_x[i,:] = data.x[mapping[i],:]
    new_y[i] = int(data.y[mapping[i]])

  data_new_1 = copy.copy(data)
  data_new_1.x = new_x
  data_new_1.y = new_y.long()
  data_new_1.num_nodes = len(new_y)
  loaded_tensor = torch.load(f"gdrive/My Drive/GNN_Sparsification/Baselines/Dspar_INDUCED/10%/10%_dir_induced_arxiv_sparsified_{epsilon:.2f}.pt")
  data_new_1.edge_index = loaded_tensor[1]

  print(epsilon)


  AVG_time_GCN = 0
  AVG_time_GAT = 0
  AVG_time_GraphSAGE = 0

  for i in range(num_of_runs):
    start_time = time.time()
    best_model_GCN.eval()
    pred = best_model_GCN(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GCN += elapsed_time


    if i == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GCN Inference time:", AVG_time_GCN/num_of_runs, "seconds")
      print(f'GCN Inference Speed-up: {Constant_GCN/(AVG_time_GCN/num_of_runs):.4f}')
      print(f'GCN Inference Accuracy: {acc:.4f}')

  for j in range(num_of_runs):
    AVG_time = 0
    start_time = time.time()
    best_model_GAT.eval()
    pred = best_model_GAT(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GAT += elapsed_time


    if j == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GAT Inference time:", AVG_time_GAT/num_of_runs, "seconds")
      print(f'GAT Inference Speed-up: {Constant_GAT/(AVG_time_GAT/num_of_runs):.4f}')
      print(f'GAT Inference Accuracy: {acc:.4f}')

  for m in range(num_of_runs):
    start_time = time.time()
    best_model_GraphSage.eval()
    pred = best_model_GraphSage(data_new_1.x, data_new_1.edge_index).argmax(dim=1)

    # End recording the current time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    AVG_time_GraphSAGE += elapsed_time



    if m == num_of_runs-1:
      correct = compute_accuracy(pred[new_indices], data_new_1.y[new_indices])
      acc = int(correct) / len(tested_elements)
      print("Average of "+str(num_of_runs) + " runs GraphSAGE Inference time:", AVG_time_GraphSAGE/num_of_runs, "seconds")
      print(f'GraphSAGE Inference Speed-up: {Constant_GraphSAGE/(AVG_time_GraphSAGE/num_of_runs):.4f}')
      print(f'GraphSAGE Inference Accuracy: {acc:.4f}')

  epsilon+=0.2


0.35
Average of 100 runs GCN Inference time: 0.268235399723053 seconds
GCN Inference Speed-up: 3.5008
GCN Inference Accuracy: 0.5631
Average of 100 runs GAT Inference time: 0.322331395149231 seconds
GAT Inference Speed-up: 3.3957
GAT Inference Accuracy: 0.5623
Average of 100 runs GraphSAGE Inference time: 0.18289923429489136 seconds
GraphSAGE Inference Speed-up: 3.2712
GraphSAGE Inference Accuracy: 0.5751
0.55
Average of 100 runs GCN Inference time: 0.2711269783973694 seconds
GCN Inference Speed-up: 3.4635
GCN Inference Accuracy: 0.5624
Average of 100 runs GAT Inference time: 0.32890404224395753 seconds
GAT Inference Speed-up: 3.3278
GAT Inference Accuracy: 0.5619
Average of 100 runs GraphSAGE Inference time: 0.18586703300476073 seconds
GraphSAGE Inference Speed-up: 3.2190
GraphSAGE Inference Accuracy: 0.5750
0.75
Average of 100 runs GCN Inference time: 0.27358327150344847 seconds
GCN Inference Speed-up: 3.4324
GCN Inference Accuracy: 0.5596
Average of 100 runs GAT Inference time: 0.33